In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [5]:
df[df['press'] == 'CoinDesk']['url'].iloc[1]

'https://www.coindesk.com/ethereum-icos-youre-wrong/'

In [7]:
coindesk = df[df['press'] == 'CoinDesk']
coindesk

,source,author,title,description,url,urlToImage,publishedAt,content,press
56,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09T23:00:16Z,New charges have been filed against a Philadel...,CoinDesk
141,"{'id': None, 'name': 'CoinDesk'}",Alyssa Hertig,Ethereum to ICOs: You're Doing It Wrong,Ethereum devs have harsh words for many ICO la...,https://www.coindesk.com/ethereum-icos-youre-w...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T09:00:52Z,"""Grotesque"" might not be the word you'd think ...",CoinDesk
167,"{'id': None, 'name': 'CoinDesk'}",Rachel Rose O'leary,Bitcoin Classic Announces Closure In Wake of S...,Proposed bitcoin scaling solution Bitcoin Clas...,https://www.coindesk.com/bitcoin-classic-annou...,https://media.coindesk.com/uploads/2017/11/clo...,2017-11-10T11:22:12Z,Proposed bitcoin scaling solution Bitcoin Clas...,CoinDesk
175,"{'id': None, 'name': 'CoinDesk'}",Sujha Sundararajan,US Treasury Secretary: We're Looking 'Carefull...,"The U.S. Secretary of the Treasury, Steven Mnu...",https://www.coindesk.com/us-treasury-secretary...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T12:05:14Z,"The U.S. Secretary of the Treasury, Steven Mnu...",CoinDesk
204,"{'id': None, 'name': 'CoinDesk'}",Omkar Godbole,Bull Grip Weakens As Bitcoin Price Breaks Belo...,Following the setting of a new record high of ...,https://www.coindesk.com/bull-grip-weakens-bit...,https://media.coindesk.com/uploads/2017/11/Cli...,2017-11-10T13:31:26Z,Bitcoin remains on the back foot ahead of the ...,CoinDesk
...,...,...,...,...,...,...,...,...,...
743942,"{'id': None, 'name': 'CoinDesk'}",Jamie Crawley,Gemini to Cease Offering Crypto Services in Ne...,The exchange said it is taking the step due to...,https://www.coindesk.com/policy/2023/09/29/gem...,https://www.coindesk.com/resizer/5JOs1BQJ79kue...,2023-09-29T12:47:11Z,"Please note that ourprivacy policy,terms of us...",CoinDesk
743960,"{'id': None, 'name': 'CoinDesk'}","Lyllah Ledesma, Omkar Godbole",First Mover Americas: Circle Argues Stablecoin...,The latest price moves in bitcoin (BTC) and cr...,https://www.coindesk.com/markets/2023/09/29/fi...,https://www.coindesk.com/resizer/iDYQEx_XIxIfl...,2023-09-29T13:10:20Z,This article originally appeared in First Move...,CoinDesk
744030,"{'id': None, 'name': 'CoinDesk'}",Danny Chong,Where Liquid Staking Meets Tokenization,"The crypto industry is changing rapidly, and l...",https://www.coindesk.com/consensus-magazine/20...,https://www.coindesk.com/resizer/Y0wciIP16Z0Yw...,2023-09-29T15:00:21Z,Liquid staking has the potential to be the bui...,CoinDesk
744047,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Valkyrie Backs Off Ether Futures Purchases Unt...,The asset manager Thursday said it had already...,https://www.coindesk.com/markets/2023/09/29/va...,https://www.coindesk.com/resizer/psYGVjbtB9QL0...,2023-09-29T15:24:23Z,"Please note that ourprivacy policy,terms of us...",CoinDesk


In [13]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import re

def get_contents(data):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(data['url']):
        try:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")
            content = soup.select(".at-content-features")

            # If the .at-content-features selector returns an empty list, try the .content selector
            if not content:
                content = soup.select(".composer-content")

            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
            contents.append(cleaned_content)
        except:
            contents.append('')  # Note the lowercase 'append' method name

    return contents


In [15]:
head = coindesk.head()

In [17]:
# Adding the content to the DataFrame
coindesk['content_full'] = get_contents(coindesk)
coindesk

100%|██████████| 21189/21189 [21:03:16<00:00,  3.58s/it]   
<ipython-input-17-acb6470e159a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coindesk['content_full'] = get_contents(coindesk)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
56,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09T23:00:16Z,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...
141,"{'id': None, 'name': 'CoinDesk'}",Alyssa Hertig,Ethereum to ICOs: You're Doing It Wrong,Ethereum devs have harsh words for many ICO la...,https://www.coindesk.com/ethereum-icos-youre-w...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T09:00:52Z,"""Grotesque"" might not be the word you'd think ...",CoinDesk,"""Grotesque"" might not be the word you'd think ..."
167,"{'id': None, 'name': 'CoinDesk'}",Rachel Rose O'leary,Bitcoin Classic Announces Closure In Wake of S...,Proposed bitcoin scaling solution Bitcoin Clas...,https://www.coindesk.com/bitcoin-classic-annou...,https://media.coindesk.com/uploads/2017/11/clo...,2017-11-10T11:22:12Z,Proposed bitcoin scaling solution Bitcoin Clas...,CoinDesk,A group of developers coding under the banner ...
175,"{'id': None, 'name': 'CoinDesk'}",Sujha Sundararajan,US Treasury Secretary: We're Looking 'Carefull...,"The U.S. Secretary of the Treasury, Steven Mnu...",https://www.coindesk.com/us-treasury-secretary...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T12:05:14Z,"The U.S. Secretary of the Treasury, Steven Mnu...",CoinDesk,"The U.S. Secretary of the Treasury, Steven Mnu..."
204,"{'id': None, 'name': 'CoinDesk'}",Omkar Godbole,Bull Grip Weakens As Bitcoin Price Breaks Belo...,Following the setting of a new record high of ...,https://www.coindesk.com/bull-grip-weakens-bit...,https://media.coindesk.com/uploads/2017/11/Cli...,2017-11-10T13:31:26Z,Bitcoin remains on the back foot ahead of the ...,CoinDesk,Bitcoin remains on the back foot ahead of the ...
...,...,...,...,...,...,...,...,...,...,...
743942,"{'id': None, 'name': 'CoinDesk'}",Jamie Crawley,Gemini to Cease Offering Crypto Services in Ne...,The exchange said it is taking the step due to...,https://www.coindesk.com/policy/2023/09/29/gem...,https://www.coindesk.com/resizer/5JOs1BQJ79kue...,2023-09-29T12:47:11Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Cryptocurrency exchange Gemini will stop offer...
743960,"{'id': None, 'name': 'CoinDesk'}","Lyllah Ledesma, Omkar Godbole",First Mover Americas: Circle Argues Stablecoin...,The latest price moves in bitcoin (BTC) and cr...,https://www.coindesk.com/markets/2023/09/29/fi...,https://www.coindesk.com/resizer/iDYQEx_XIxIfl...,2023-09-29T13:10:20Z,This article originally appeared in First Move...,CoinDesk,This article originally appeared in First Move...
744030,"{'id': None, 'name': 'CoinDesk'}",Danny Chong,Where Liquid Staking Meets Tokenization,"The crypto industry is changing rapidly, and l...",https://www.coindesk.com/consensus-magazine/20...,https://www.coindesk.com/resizer/Y0wciIP16Z0Yw...,2023-09-29T15:00:21Z,Liquid staking has the potential to be the bui...,CoinDesk,Liquid staking has the potential to be the bui...
744047,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Valkyrie Backs Off Ether Futures Purchases Unt...,The asset manager Thursday said it had already...,https://www.coindesk.com/markets/2023/09/29/va...,https://www.coindesk.com/resizer/psYGVjbtB9QL0...,2023-09-29T15:24:23Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Valkyrie Friday morning said it will not purch...


In [19]:
coindesk[coindesk['content_full'] == '']

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1453,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,UK Hedge Fund Eyes Cryptocurrencies On Heels o...,British hedge fund Man Group's CEO Luke Ellis ...,https://www.coindesk.com/man-group-ceo-hedge-f...,https://media.coindesk.com/uploads/2017/09/Unt...,2017-11-15T07:00:01Z,"The CEO of the Man Group, a major U.K.-based h...",CoinDesk,
2834,"{'id': None, 'name': 'CoinDesk'}",Daniel Palmer,Bitcoin Price Reaches New Record High Close to...,As the cryptocurrency market in general contin...,https://www.coindesk.com/bitcoin-price-reaches...,https://media.coindesk.com/uploads/2017/11/Flo...,2017-11-20T08:30:15Z,As the cryptocurrency market continues to clim...,CoinDesk,
5617,"{'id': None, 'name': 'CoinDesk'}",Nolan Bauerle,"State of Blockchain Q3 2017: Crypto Assets, Ba...",CoinDesk has released its latest quarterly rep...,https://www.coindesk.com/coindesk-state-of-blo...,https://media.coindesk.com/uploads/2017/11/Scr...,2017-11-28T11:00:23Z,Nolan Bauerle is a former researcher and write...,CoinDesk,
7198,"{'id': None, 'name': 'CoinDesk'}",Aaron Stanley,"Questions of Regulation, Taxation Loom Over Cr...",As the world of cryptocurrencies and ICOs heat...,https://www.coindesk.com/questions-of-regulati...,https://media.coindesk.com/uploads/2017/11/201...,2017-11-30T13:00:04Z,"The legacy of the W.J. Howey Company lives on,...",CoinDesk,
8418,"{'id': None, 'name': 'CoinDesk'}",Brady Dale,7 Tokens Investors Are Talking About,Disclaimer: This article should not be taken a...,https://www.coindesk.com/boost-compound-pantera/,https://media.coindesk.com/uploads/2017/11/lig...,2017-12-02T16:00:08Z,Disclaimer: This article should not be taken a...,CoinDesk,
...,...,...,...,...,...,...,...,...,...,...
681308,"{'id': None, 'name': 'CoinDesk'}",Matt Kolesky,Where the SEC’s Proposed Custody Rule Comes Up...,Financial advisors looking for clarity on cryp...,https://www.coindesk.com/coindesk-indices/2023...,https://www.coindesk.com/resizer/VW8HKduMaF8Rv...,2023-05-18T15:17:22Z,"In February, the SEC proposed dramatic changes...",CoinDesk,
684526,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Helium Expands IoT Coverage Through Amazon Sid...,Amazon Sidewalk is a new nationwide network us...,https://www.coindesk.com/business/2023/05/25/h...,https://www.coindesk.com/resizer/vmaQkte69FBYR...,2023-05-25T13:00:00Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,
684773,"{'id': None, 'name': 'CoinDesk'}",Ian Allison,Alameda-Backed ‘Samcoins’ CEO Alex Grebnev Sue...,"Alex Grebnev, CEO of the Maps.me (MAPs) and Ox...",https://www.coindesk.com/policy/2023/05/25/ala...,https://www.coindesk.com/resizer/sIT2_q31taQxt...,2023-05-25T20:00:32Z,"Alex Grebnev, CEO of the Maps.me (MAPs) and Ox...",CoinDesk,
713856,"{'id': None, 'name': 'CoinDesk'}",Shaurya Malwa,"XRP, Stellar and Solana Products See Increased...",Products focused on XLM saw nearly 63% higher ...,https://www.coindesk.com/markets/2023/07/27/xr...,https://www.coindesk.com/resizer/a2dfdNnfziX_3...,2023-07-27T09:01:34Z,Institutional investors increased allocations ...,CoinDesk,


In [23]:
empty_rows = coindesk[coindesk['content_full'] == '']

empty_rows['url'].iloc[700]

'https://www.coindesk.com/market-wrap-bitcoin-34k-ether-futures-jump-350'

In [24]:
coindesk

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
56,"{'id': None, 'name': 'CoinDesk'}",Nikhilesh De,More Charges Filed Against Trader Who Used Bit...,A Philadelphia day trader was formally indicte...,https://www.coindesk.com/day-trader-formally-i...,https://media.coindesk.com/uploads/2017/11/Unt...,2017-11-09T23:00:16Z,New charges have been filed against a Philadel...,CoinDesk,New charges have been filed against a Philadel...
141,"{'id': None, 'name': 'CoinDesk'}",Alyssa Hertig,Ethereum to ICOs: You're Doing It Wrong,Ethereum devs have harsh words for many ICO la...,https://www.coindesk.com/ethereum-icos-youre-w...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T09:00:52Z,"""Grotesque"" might not be the word you'd think ...",CoinDesk,"""Grotesque"" might not be the word you'd think ..."
167,"{'id': None, 'name': 'CoinDesk'}",Rachel Rose O'leary,Bitcoin Classic Announces Closure In Wake of S...,Proposed bitcoin scaling solution Bitcoin Clas...,https://www.coindesk.com/bitcoin-classic-annou...,https://media.coindesk.com/uploads/2017/11/clo...,2017-11-10T11:22:12Z,Proposed bitcoin scaling solution Bitcoin Clas...,CoinDesk,A group of developers coding under the banner ...
175,"{'id': None, 'name': 'CoinDesk'}",Sujha Sundararajan,US Treasury Secretary: We're Looking 'Carefull...,"The U.S. Secretary of the Treasury, Steven Mnu...",https://www.coindesk.com/us-treasury-secretary...,https://media.coindesk.com/uploads/2017/11/shu...,2017-11-10T12:05:14Z,"The U.S. Secretary of the Treasury, Steven Mnu...",CoinDesk,"The U.S. Secretary of the Treasury, Steven Mnu..."
204,"{'id': None, 'name': 'CoinDesk'}",Omkar Godbole,Bull Grip Weakens As Bitcoin Price Breaks Belo...,Following the setting of a new record high of ...,https://www.coindesk.com/bull-grip-weakens-bit...,https://media.coindesk.com/uploads/2017/11/Cli...,2017-11-10T13:31:26Z,Bitcoin remains on the back foot ahead of the ...,CoinDesk,Bitcoin remains on the back foot ahead of the ...
...,...,...,...,...,...,...,...,...,...,...
743942,"{'id': None, 'name': 'CoinDesk'}",Jamie Crawley,Gemini to Cease Offering Crypto Services in Ne...,The exchange said it is taking the step due to...,https://www.coindesk.com/policy/2023/09/29/gem...,https://www.coindesk.com/resizer/5JOs1BQJ79kue...,2023-09-29T12:47:11Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Cryptocurrency exchange Gemini will stop offer...
743960,"{'id': None, 'name': 'CoinDesk'}","Lyllah Ledesma, Omkar Godbole",First Mover Americas: Circle Argues Stablecoin...,The latest price moves in bitcoin (BTC) and cr...,https://www.coindesk.com/markets/2023/09/29/fi...,https://www.coindesk.com/resizer/iDYQEx_XIxIfl...,2023-09-29T13:10:20Z,This article originally appeared in First Move...,CoinDesk,This article originally appeared in First Move...
744030,"{'id': None, 'name': 'CoinDesk'}",Danny Chong,Where Liquid Staking Meets Tokenization,"The crypto industry is changing rapidly, and l...",https://www.coindesk.com/consensus-magazine/20...,https://www.coindesk.com/resizer/Y0wciIP16Z0Yw...,2023-09-29T15:00:21Z,Liquid staking has the potential to be the bui...,CoinDesk,Liquid staking has the potential to be the bui...
744047,"{'id': None, 'name': 'CoinDesk'}",Lyllah Ledesma,Valkyrie Backs Off Ether Futures Purchases Unt...,The asset manager Thursday said it had already...,https://www.coindesk.com/markets/2023/09/29/va...,https://www.coindesk.com/resizer/psYGVjbtB9QL0...,2023-09-29T15:24:23Z,"Please note that ourprivacy policy,terms of us...",CoinDesk,Valkyrie Friday morning said it will not purch...


In [25]:
coindesk.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\coindesk_5.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content